In [56]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [57]:
# This cell should be edited to match the data to be inspected.

# Which calibration type to analyze.
calibType = 'cpPtcExtract' 
# Camera
cameraName = 'LSSTComCam'
numberDetectors = 9

# Set which display to use.
afwDisplay.setDefaultBackend("matplotlib")



In [59]:
# Collection name containing the verification outputs. See DM-34867
# Low flux: 2708.223443552568
verifyCollection = "u/plazas/DM-34867-2022SEP26-VERIFY-gain.comcam.FULLCOV.5" 
# Collection name containign the PTC dataset
genCollection = "u/plazas/2022SEP20-ptc.comcam.DM-34867-FULLCOV.1" 

In [60]:
# Get butler and camera
# the genCollection has the camera
butler = dB.Butler("/repo/main/", collections=[genCollection,verifyCollection])
camera = butler.get('camera', instrument=cameraName)

In [61]:
# Get Run Statistics
runStats = butler.get('verifyGainStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')


In [62]:
runSuccess


False

In [63]:
runStats

{'Detector 0': {'FAILURES': ['Detector 0 GAIN_FROM_FLAT_PAIR',
   'Detector 0 ISR_NOISE']},
 'Detector 1': {'FAILURES': ['Detector 1 GAIN_FROM_FLAT_PAIR',
   'Detector 1 ISR_NOISE']},
 'Detector 2': {'FAILURES': ['Detector 2 GAIN_FROM_FLAT_PAIR',
   'Detector 2 ISR_NOISE']},
 'Detector 3': {'FAILURES': ['Detector 3 GAIN_FROM_FLAT_PAIR',
   'Detector 3 ISR_NOISE']},
 'Detector 4': {'FAILURES': ['Detector 4 GAIN_FROM_FLAT_PAIR',
   'Detector 4 ISR_NOISE']},
 'Detector 5': {'FAILURES': ['Detector 5 GAIN_FROM_FLAT_PAIR',
   'Detector 5 ISR_NOISE']},
 'Detector 6': {'FAILURES': ['Detector 6 ISR_NOISE']},
 'Detector 7': {'FAILURES': ['Detector 7 GAIN_FROM_FLAT_PAIR',
   'Detector 7 ISR_NOISE']},
 'Detector 8': {'FAILURES': ['Detector 8 GAIN_FROM_FLAT_PAIR',
   'Detector 8 ISR_NOISE']}}

In [64]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)



Detector 0,Detector,1,1
Detector 1,Detector,1,1
Detector 2,Detector,1,1
Detector 3,Detector,1,1
Detector 4,Detector,1,1
Detector 5,Detector,1,1
Detector 6,Detector,1,0
Detector 7,Detector,1,1
Detector 8,Detector,1,1


In [65]:
# Let's look at the values to understand bette rthe table from above.
# calibType with gain and ISR noise is `cpPtcExtract`, from the gain-from-flat-pairs subset in the cpPtc.yaml
gainThreshold = 5  # percentage, default value in config
noiseThreshold = 5 # percentage, default value in config
for detId in range(9):
    calib = butler.get(calibType, instrument="LSSTComCam", detector=detId, exposure=2022021800078)
    detector = camera[detId]
    print (f"Detector ID: {detId}")
    for amp in detector:
        ampName = amp.getName()
        print (f"    {ampName}")
        diffGain = 100* (np.abs(calib.gain[ampName] - amp.getGain()) / amp.getGain())
        diffNoise = 100* (np.abs(calib.noise[ampName] - amp.getReadNoise()) / amp.getReadNoise())
        testGain = bool(diffGain > gainThreshold)
        testNoise = bool(diffNoise > noiseThreshold)
        print ( f"Noise --  Difference(%): {diffNoise}, ISR noise: {calib.noise[ampName]}, Amp. Noise {amp.getReadNoise()}")
        print ( f"Gain  --  Difference(%): {diffGain}, Gain from pair: {calib.gain[ampName]}, Amp. gain {amp.getGain()}")
        print (" ")
    print(" ")
    print(" ")
    

Detector ID: 0
    C10
Noise --  Difference(%): 7.296073857967578, ISR noise: 6.48927482994227, Amp. Noise 7.0
Gain  --  Difference(%): 11.509788117657742, Gain from pair: 1.5043336019998184, Amp. gain 1.7
 
    C11
Noise --  Difference(%): 6.054066889218542, ISR noise: 7.423784682245298, Amp. Noise 7.0
Gain  --  Difference(%): 59.49064144782618, Gain from pair: 0.6886590953869549, Amp. gain 1.7
 
    C12
Noise --  Difference(%): 12.99603552158263, ISR noise: 7.909722486510784, Amp. Noise 7.0
Gain  --  Difference(%): 3.830164527641015, Gain from pair: 1.6348872030301027, Amp. gain 1.7
 
    C13
Noise --  Difference(%): 13.698735346228045, ISR noise: 7.958911474235963, Amp. Noise 7.0
Gain  --  Difference(%): 9.430796767756215, Gain from pair: 1.5396764549481443, Amp. gain 1.7
 
    C14
Noise --  Difference(%): 14.330614075578627, ISR noise: 8.003142985290504, Amp. Noise 7.0
Gain  --  Difference(%): 4.081298424652782, Gain from pair: 1.6306179267809027, Amp. gain 1.7
 
    C15
Noise --  

In [73]:
# Gain, ISR noise and other values from 'cpPtcExtract' are saved as 'AMP' statistics
for detId in range(numberDetectors):
    detStats = butler.get("verifyGainDetStats", instrument="LSSTComCam", detector=detId)
    print ("Detector ID: ", detId) 
    print ("    Overall success: ", detStats['SUCCESS'])
    print ("    Overall success per test type: ", detStats['VERIFY'])
    for ampName in detStats['AMP']:
        print (f"    {ampName}", detStats['AMP'][ampName])
    print (" ")

Detector ID:  0
    Overall success:  False
    Overall success per test type:  {'GAIN_FROM_FLAT_PAIR': False, 'ISR_NOISE': False}
    C00 {'AMP_GAIN': 1.7, 'AMP_NOISE': 7.0, 'GAIN_CORRECTION_FACTOR': 0.9934304550480343, 'GAIN_FROM_FLAT_PAIR': 1.7186524041090097, 'ISR_NOISE': 6.301029864596574, 'MEAN_FLUX_FLAT_PAIR': 2433.1647970243503}
    C01 {'AMP_GAIN': 1.7, 'AMP_NOISE': 7.0, 'GAIN_CORRECTION_FACTOR': 0.9933727716691528, 'GAIN_FROM_FLAT_PAIR': 1.7025602854501152, 'ISR_NOISE': 6.78142552641049, 'MEAN_FLUX_FLAT_PAIR': 2454.5290114248855}
    C02 {'AMP_GAIN': 1.7, 'AMP_NOISE': 7.0, 'GAIN_CORRECTION_FACTOR': 0.9933035589760368, 'GAIN_FROM_FLAT_PAIR': 1.7013213747134432, 'ISR_NOISE': 7.3298436041324315, 'MEAN_FLUX_FLAT_PAIR': 2480.1633422085933}
    C03 {'AMP_GAIN': 1.7, 'AMP_NOISE': 7.0, 'GAIN_CORRECTION_FACTOR': 0.993277007929838, 'GAIN_FROM_FLAT_PAIR': 1.7264755747545433, 'ISR_NOISE': 7.704248769007972, 'MEAN_FLUX_FLAT_PAIR': 2489.997063022964}
    C04 {'AMP_GAIN': 1.7, 'AMP_NOISE': 